
## Imports

In [93]:
%reload_ext autoreload
%autoreload 2
from merkle_tree import MerkleTree
from channel import Channel
from hash import my_hash
from pprint import pprint

In [94]:
# Test

from pprint import pprint

m = MerkleTree([1, 2, 3, 4, 5, 6, 7, 8])
pprint([[b[:4] for b in a] for a in m.tree])
print(m.root)
pprint([a[:4] for a in m.get_path(0)])

[['6b86', 'd473', '4e07', '4b22', 'ef2d', 'e7f6', '7902', '2c62'],
 ['33b6', '1365', '4358', 'ada1'],
 ['85df', 'e0e2'],
 ['c274']]
c27450cd3fd4df029145f3437ae9c381e0ae55e8400de06cb973005b36d7b222
['6b86',
 ('d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35', 'right'),
 ('13656c83d841ea7de6ebf3a89e0038fea9526bd7f686f06f7a692343a8a32dca', 'right'),
 ('e0e2d0cec0ef7e8fc458e516dfde82890c183431a3f9efae9e4693fc23dfa36a', 'right')]


### Fibonacci function

In [95]:
def fibonacci(a=1, size=1001) -> list[int]:
    fib_list = [1, a]
    for _ in range(2, size):
        fib_list.append(fib_list[-1] + fib_list[-2])
    return fib_list

In [96]:
print(fibonacci())

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074, 32951280099, 53316291173, 86267571272, 139583862445, 225851433717, 365435296162, 591286729879, 956722026041, 1548008755920, 2504730781961, 4052739537881, 6557470319842, 10610209857723, 17167680177565, 27777890035288, 44945570212853, 72723460248141, 117669030460994, 190392490709135, 308061521170129, 498454011879264, 806515533049393, 1304969544928657, 2111485077978050, 3416454622906707, 5527939700884757, 8944394323791464, 14472334024676221, 23416728348467685, 37889062373143906, 61305790721611591, 99194853094755497, 160500643816367088, 259695496911122585, 420196140727489673, 679891637638612258, 110008777836

### Polynomial f(x)

In [97]:
P: int =3221225473
F = GF(P)
N = 8192

g = F(1734477367)
gamma = g ** 8
gamma_group = [gamma ** i for i in range(1001)]

fibonacci_seq = fibonacci()

R.<x> = PolynomialRing(F)

f = R.lagrange_polynomial(zip(gamma_group, fibonacci_seq))

### Bit Reverse Order

In [98]:
def bit_reverse(n, width):
    rev = 0  # This will store the reversed bits
    for i in range(width):
        rev = (rev << 1) | (n & 1)  # Shift left and take LSB of n
        n >>= 1  # Shift n to the right
    return rev


def bit_reverse_permutation(N):
    """Generate bit-reversed order for N elements."""
    width = int(math.log(N, 2))
    return [bit_reverse(i, width) for i in range(N)]


bit_reverse_order = bit_reverse_permutation(N)

### w < g >

In [99]:
w = F.multiplicative_generator()

g_group = [g ** i for i in range(N)]

wg = [w * gi for gi in g_group]
wg_merkle_order = [wg[i] for i in bit_reverse_order]

### Commit f(x) on LDE

In [100]:
channel = Channel(F, N)

f_wg = [f(x) for x in wg_merkle_order]

merkle_f_wg = MerkleTree(f_wg)

channel.send({'title': 'f(x) on LDE', 'data': merkle_f_wg.root})

### Polynomial h(x)

In [101]:
d_x = 1
for gamma_i in gamma_group[:-2]:
    d_x = d_x * (x - gamma_i)

F_x = f(x) + f(gamma * x) - f((gamma**2) * x)

h_x = F_x // d_x

In [102]:
h_wg = [F_x(x) / d_x(x) for x in wg_merkle_order]

# Sanity check
h_wg_by_poly = [h_x(x) for x in wg_merkle_order]
print(h_wg_by_poly[:10])
print(h_wg[:10])
assert(h_wg == h_wg_by_poly)

[1860826706, 1845649622, 290025805, 195225050, 2156032599, 1550443729, 1756057239, 1950419089, 356807959, 128442896]
[1860826706, 1845649622, 290025805, 195225050, 2156032599, 1550443729, 1756057239, 1950419089, 356807959, 128442896]


### Boundary Constraints

In [103]:
Y = fibonacci_seq[-1]

t0_x = (f(x) - 1) / (x - gamma ** 0)
t1_x = (f(x) - Y) / (x - gamma ** 1000)

In [104]:
t0_wg = [(f(i) - 1) / (i - gamma ** 0) for i in wg_merkle_order]
t1_wg = [(f(i) - Y) / (i - gamma ** 1000) for i in wg_merkle_order]

# Sanity check
t0_x_by_poly = [t0_x(x) for x in wg_merkle_order]
t1_x_by_poly = [t1_x(x) for x in wg_merkle_order]
print(t0_x_by_poly[:10])
print(t0_wg[:10])
print("")
print(t1_x_by_poly[:10])
print(t1_wg[:10])
assert(t0_wg == t0_x_by_poly)
assert(t1_wg == t1_x_by_poly)

[2384530853, 599226013, 2071111314, 1402510381, 2026908630, 951059204, 1906847992, 1092764146, 3083267015, 51241944]
[2384530853, 599226013, 2071111314, 1402510381, 2026908630, 951059204, 1906847992, 1092764146, 3083267015, 51241944]

[2395950328, 2696799910, 124207998, 3010185695, 1771367485, 510712279, 521057469, 2771188778, 1200373265, 956436872]
[2395950328, 2696799910, 124207998, 3010185695, 1771367485, 510712279, 521057469, 2771188778, 1200373265, 956436872]


### Composition Polynomial

In [105]:
beta_0 = channel.receive_random_field_element()
beta_1 = channel.receive_random_field_element()
beta_2 = channel.receive_random_field_element()

cp0_x = beta_0 * h_x + beta_1 * t0_x + beta_2 * t1_x

In [106]:
cp0_wg = [beta_0 * h_wg[i] + beta_1 * t0_wg[i] + beta_2 * t1_wg[i]
          for i in range(len(wg))]

# Sanity check
cp0_x_by_poly = [cp0_x(x) for x in wg_merkle_order]
print(cp0_x_by_poly[:10])
print(cp0_wg[:10])
assert(cp0_wg == cp0_x_by_poly)

[505489157, 2368666279, 1774153425, 2341526794, 1377743153, 1534405270, 1449131553, 978573396, 2713377995, 1641628493]
[505489157, 2368666279, 1774153425, 2341526794, 1377743153, 1534405270, 1449131553, 978573396, 2713377995, 1641628493]


### FRI

In [107]:
fri_layers = []
degree = R(cp0_x).degree()
# if degree % 2 != 0:
#     degree += 1

curr_cp = [cp0_x(x) for x in wg_merkle_order]
while degree > 0:
    # Commit curr layer
    merkle_curr_cp = MerkleTree(curr_cp)
    fri_layers.append((curr_cp, merkle_curr_cp))
    channel.send({
        "title": f"commit for layer {len(fri_layers) - 1} of FRI",
        "data": merkle_curr_cp.root,
    })
    query_x = channel.receive_random_field_element()
    curr_cp = [((curr_cp[i] + curr_cp[i + 1]) / 2) +
               query_x * (curr_cp[i] - curr_cp[i + 1]) / (2 * wg_merkle_order[i]) for i in range(len(curr_cp))[::2]]
    degree = degree // 2

constant = curr_cp[0]
channel.send({"title:": "constant of last FRI layer",
              "data": str(constant),
              })

assert(all([constant == x for x in curr_cp]))

In [108]:
print(fri_layers)
print("Merkle Root")
print(curr_cp[0])

[([505489157, 2368666279, 1774153425, 2341526794, 1377743153, 1534405270, 1449131553, 978573396, 2713377995, 1641628493, 1816342573, 1289231739, 163652991, 3163998221, 3129458324, 654807016, 1703307083, 2745128427, 1668637507, 431908585, 506127014, 1328804926, 1516771919, 1972204366, 318889344, 905308738, 568388212, 33178379, 2970801354, 3046060680, 2540446579, 1044007360, 820685235, 693802886, 368457931, 2777450652, 1939350136, 2705786313, 585386390, 504146863, 2354513796, 2297613773, 2536080431, 2032817436, 1424986626, 1466538727, 2839230967, 1027563590, 2690837352, 2044944558, 1052392364, 3124654194, 687305723, 634950103, 1211233220, 2001756531, 3039466063, 588508465, 341617448, 66159863, 3164048094, 2862222386, 1228742198, 2633373344, 1921700180, 2069312452, 1370023042, 1154011977, 1297837542, 2277741333, 367993461, 2455720301, 1608268153, 2917908630, 2533093575, 2762865717, 111252872, 1738496015, 1601955947, 245722976, 152421473, 2104186256, 649257575, 2399543857, 2321250688, 2109

In [109]:
# Sanity check
print(curr_cp)

[1527770659, 1527770659, 1527770659, 1527770659, 1527770659, 1527770659, 1527770659, 1527770659]


In [110]:
pprint(channel.get_all_messages())

[{'data': 'd524fba68e1263f084afe501c715dca773c616d5a8c52303322985d32c94ea56',
  'title': 'f(x) on LDE'},
 {'data': 2415762890, 'title': 'Random Field Element'},
 {'data': 1173033395, 'title': 'Random Field Element'},
 {'data': 519019791, 'title': 'Random Field Element'},
 {'data': '1a10f17d9c3fd15f47317a3a8620d4faabd3e7a2010a9a7f5e7905b67f417c3b',
  'title': 'commit for layer 0 of FRI'},
 {'data': 2052863614, 'title': 'Random Field Element'},
 {'data': '02622deed36aa502ef6123b45d12be300de1ea91ad247678fba50c451b5fa7ec',
  'title': 'commit for layer 1 of FRI'},
 {'data': 33657231, 'title': 'Random Field Element'},
 {'data': '45f9b3f6a707262336d628878d361f5a6ad4cf47137a1524bdf700e32bbc2679',
  'title': 'commit for layer 2 of FRI'},
 {'data': 2459535427, 'title': 'Random Field Element'},
 {'data': '28a5d1c2c6e53aeec1e7d177ece2ae89c86ba65ae1174604de73a6051999059f',
  'title': 'commit for layer 3 of FRI'},
 {'data': 1036729176, 'title': 'Random Field Element'},
 {'data': 'ff2cfbac504d1ef4a6c

### Decommit Phase

# TIOTA


## Decommit Phase

In [111]:
def send_f_of_x_and_gammas(idx_of_x, idx_of_gamma_x, idx_of_gamma_2_x):
    channel.send({
        'title': 'f of x',
        'data': f_wg[idx_of_x],
    })
    channel.send({
        'title': 'path of f of x',
        'data': merkle_f_wg.get_path(idx_of_x),
    })
    channel.send({
        'title': 'f of gamma x',
        'data': f_wg[idx_of_gamma_x],
    })
    channel.send({
        'title': 'path of f of gamma x',
        'data': merkle_f_wg.get_path(idx_of_gamma_x),
    })
    channel.send({
        'title': 'f of gamma squared x',
        'data': f_wg[idx_of_gamma_2_x],
    })
    channel.send({
        'title': 'path of f of gamma squared x',
        'data': merkle_f_wg.get_path(idx_of_gamma_2_x),
    })

def send_cp_layer(idx_of_x, idx_of_minus_x, cp, cp_merkle, i):
    assert(cp_merkle.get_path(idx_of_x)[2:] == cp_merkle.get_path(idx_of_minus_x)[2:])
    channel.send({
        'title': f'path of cp of x - layer {i}',
        'data': cp_merkle.get_path(idx_of_x),
    })
    channel.send({
        'title': f'cp of -x - layer {i}',
        'data': cp[idx_of_minus_x],
    })
    channel.send({
        'title': f'path of cp of -x - layer {i}',
        'data': cp_merkle.get_path(idx_of_minus_x),
    })

def query(idx):
    idx_of_x = bit_reverse_order.index(idx)
    idx_of_gamma_x = bit_reverse_order.index(idx + 8)
    idx_of_gamma_2_x = bit_reverse_order.index(idx + 16)
    send_f_of_x_and_gammas(idx_of_x, idx_of_gamma_x, idx_of_gamma_2_x)
    for i, (cp, cp_merkle) in enumerate(fri_layers):
        idx_of_minus_x = idx_of_x ^^ 1
        send_cp_layer(idx_of_x, idx_of_minus_x, cp, cp_merkle, i)
        idx_of_x //= 2

In [112]:
channel.start_decommit()
for i in range(8):
    random_idx = channel.receive_random_query()
    query(random_idx)

In [113]:
def check_merkle_path(data, path, root):
    curr = my_hash(data)
    if curr != path[0]:
        return False
    for sibling, direction in path[1:]:
        if direction == 'left':
            curr = my_hash(sibling + curr)
        else:
            curr = my_hash(curr + sibling)
    return curr == root

# Sanity
assert(check_merkle_path(f_wg[4000], merkle_f_wg.get_path(4000), merkle_f_wg.root))

# Verifier

In [114]:
channel.receive_decommitment()

[{'title': 'Random Query', 'data': 7633},
 {'title': 'f of x', 'data': 2305929754},
 {'title': 'path of f of x',
  'data': ['3805376aea5b375375cf57a62f53f57f43a570713a4817239df5d4eb49948320',
   ('45132041f7bdd5557fbf34b971a4ca0d5caf75633fb6dfbd918d276f0b1f02d0',
    'left'),
   ('0da6bd26507c8f5c9f989a59797f399c3ce6877604581f3b507de50ea28d7669',
    'left'),
   ('c3dda4e2871cb2fa5b01901a1370ea1a76cf3d3815d261e5ffc87919b0b63f3c',
    'left'),
   ('fba6e3e83e000da360a27845359f53b17516045f8f5572268d70f722d51fe221',
    'right'),
   ('03de03f12879e010d05f73ad1e95f2233a8118f6656f216c680dc01e28654bd6',
    'left'),
   ('229172c00dc0f591a3adf1f91d2674c55177d4f12fbeec68f261ae4352124f2b',
    'left'),
   ('94ebce9bea64a99c4b43e0020293ec9759f142edbcee13fc1f0a422908165824',
    'left'),
   ('2053ccea46c9efb800f571d6c7eb825b7436dc4a77519f94293620b3d5b72f20',
    'right'),
   ('7dd3ddec50be76c48aa39cc215a8c1300bd3766d9a3b7c2622348143c4b7c9b4',
    'left'),
   ('54810406f836cf015fd8d28389e161c3a5af

In [ ]:
P: int = 3221225473
F = GF(P)
N = 8192
g = F(1734477367)
gamma = g ** 8
Y = fibonacci_seq[-1]

# Initial tree
commitment = channel.receive_commitment()
decommitment = channel.receive_decommitment()
random_idx = decommitment[0]['data']
f_x = decommitment[1]['data']
path_of_f_x = decommitment[2]['data']
f_gamma_x = decommitment[3]['data']
path_of_f_gamma_x = decommitment[4]['data']
f_gamma_2_x = decommitment[5]['data']
path_of_f_gamma_2_x = decommitment[6]['data']
root = commitment[0]['data']
assert(check_merkle_path(f_x, path_of_f_x, root))
assert(check_merkle_path(f_gamma_x, path_of_f_gamma_x, root))
assert(check_merkle_path(f_gamma_2_x, path_of_f_gamma_2_x, root))

# CP 0
beta_0 = commitment[1]['data']
beta_1 = commitment[2]['data']
beta_2 = commitment[3]['data']

F_x = f_x + f_gamma_x - f_gamma_2_x
g_pow = g ** random_idx
query_x = w * g_pow
d_x = 1
for gamma_i in [gamma ** i for i in range(999)]:
    d_x = d_x * (query_x - gamma_i)

h_x = F_x / d_x
t0_x = (f_x - 1) / (query_x - gamma ** 0)
t1_x = (f_x - Y) / (query_x - gamma ** 1000)
cp_x = beta_0 * h_x + beta_1 * t0_x + beta_2 * t1_x

commitment_idx = 4
decommitment_idx = 7
while commitment_idx < len(commitment) - 1:
    path_of_cp_x = decommitment[decommitment_idx]['data']
    cp_minus_x = decommitment[decommitment_idx + 1]['data']
    path_of_cp_minus_x = decommitment[decommitment_idx + 2]['data']

    cp_root = commitment[commitment_idx]['data']
    assert(check_merkle_path(cp_x, path_of_cp_x, cp_root)), (cp_x, path_of_cp_x, cp_root)
    assert(check_merkle_path(cp_minus_x, path_of_cp_minus_x, cp_root))

    random_x = commitment[commitment_idx + 1]['data']
    query_x = w * g_pow
    cp_x = (cp_x + cp_minus_x) / 2 + random_x * (cp_x - cp_minus_x) / (2 * query_x)
    
    g_pow = g_pow ** 2

    commitment_idx += 2
    decommitment_idx += 3